In [1]:
#Imports and other functions required to run this file
import pandas as pd
import numpy as np
import math as math


file_import = "purchase_data.json"
nerds = pd.read_json(path_or_buf=file_import, orient=None, typ='frame', dtype=True, convert_axes=True, convert_dates=True)
#nerds.to_excel("nerds.xlsx", index=False)
#nerds.head()

In [2]:
#Count of unique Screen Names
players_df= len(nerds["SN"].unique())

count_players= pd.DataFrame([{"Total Players":players_df}])
count_players.set_index('Total Players', inplace = True)
count_players

""
Total Players
573


In [3]:
# Pricing Analysis
item_name= nerds.drop_duplicates(["Item ID"], keep = "last")
item_name = len(item_name)
avg_price= nerds["Price"].mean()
avg_price_d= "${:,.2f}".format(avg_price)
total_rev= nerds["Price"].sum()
total_rev_d= "${:,.2f}".format(total_rev)
item_id= nerds["Item ID"].count()
p_columns= ["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]
pricing={"Number of Unique Items":[item_name],
         "Average Price":[avg_price_d],
         "Total Revenue": [total_rev_d],
        "Number of Purchases":[item_id]}
pricing=pd.DataFrame.from_dict(pricing)
pricing=pricing.reindex(columns=p_columns)

pricing

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,780,"$2,286.33"


In [4]:
#Gender Demographics
g_data = nerds.drop_duplicates(["SN"], keep="last")
g_counts =g_data["Gender"].value_counts().reset_index()
g_counts["Percent of Players"]=(g_counts["Gender"]/g_counts["Gender"].sum())*100
g_counts.rename(columns={"index":"Gender", "Gender":"Total Players"},inplace=True)
g_counts.set_index(["Gender"], inplace = True)
g_c_format = g_counts
g_c_format.style.format({"Percent of Players": "{:.2f}%"})

,Total Players,Percent of Players
Gender,,
Male,465,81.15%
Female,100,17.45%
Other / Non-Disclosed,8,1.40%


In [5]:
#Purchasing Analysis by Gender
p_c_gender= pd.DataFrame(nerds.groupby("Gender")["Gender"].count())
p_p_gender= pd.DataFrame(nerds.groupby("Gender")["Price"].sum())
g_nerds = pd.merge(p_c_gender,p_p_gender, left_index=True, right_index=True)
g_nerds = g_nerds.merge(g_counts, left_index=True, right_index=True)
g_nerds["Average Price"]= g_nerds["Price"]/g_nerds["Gender"]
g_nerds["Normalized Price"]= g_nerds["Price"]/g_nerds["Total Players"]
g_nerds.rename(columns={"Gender": "Purchase Count","Price":"Total Purchase Value"}, inplace= True)
del g_nerds["Total Players"]
del g_nerds["Percent of Players"]

g_nerds
g_n_format = g_nerds
g_n_format.style.format({"Average Price": "${:.2f}", "Total Purchase Value": "${:,.2f}", "Normalized Price":"${:.2f}"})

,Purchase Count,Total Purchase Value,Average Price,Normalized Price
Gender,,,,
Female,136,$382.91,$2.82,$3.83
Male,633,"$1,867.68",$2.95,$4.02
Other / Non-Disclosed,11,$35.74,$3.25,$4.47


In [6]:
#Age Demographics
nerds.loc[(nerds["Age"] < 10), "Age Ranges"] = "<10"
nerds.loc[(nerds["Age"] >= 10) & (nerds["Age"] <= 14), "Age Ranges"] = "10 - 14"
nerds.loc[(nerds["Age"] >= 15) & (nerds["Age"] <= 19), "Age Ranges"] = "15 - 19"
nerds.loc[(nerds["Age"] >= 20) & (nerds["Age"] <= 24), "Age Ranges"] = "20 - 24"
nerds.loc[(nerds["Age"] >= 25) & (nerds["Age"] <= 29), "Age Ranges"] = "25 - 29"
nerds.loc[(nerds["Age"] >= 30) & (nerds["Age"] <= 34), "Age Ranges"] = "30 - 34"
nerds.loc[(nerds["Age"] >= 35) & (nerds["Age"] <= 39), "Age Ranges"] = "35 - 39"
nerds.loc[(nerds["Age"] >= 40), "Age Ranges"] = "> 40"

age_demo = nerds.drop_duplicates(["SN"], keep="last")
age_b = pd.DataFrame(age_demo.groupby("Age Ranges")["SN"].count())
age_b["Percentage of Players"] = (age_b["SN"]/age_b["SN"].sum())*100

age_b.rename(columns={"SN":"Total Count"},inplace=True)
age_b_format = age_b
age_b_format.style.format({"Percentage of Players": "{:.2f}%"})

,Total Count,Percentage of Players
Age Ranges,,
10 - 14,23,4.01%
15 - 19,100,17.45%
20 - 24,259,45.20%
25 - 29,87,15.18%
30 - 34,47,8.20%
35 - 39,27,4.71%
<10,19,3.32%
> 40,11,1.92%


In [12]:
#Age Purchases
age_items = pd.DataFrame(nerds.groupby("Age Ranges")["Item ID"].count())
age_purchases = pd.DataFrame(nerds.groupby("Age Ranges")["Price"].sum())
age_p_a= pd.merge(age_purchases,age_items, left_index=True, right_index=True)

age_uniq= pd.merge(age_p_a,age_b,left_index=True,right_index=True)
purch_total=age_p_a["Item ID"].sum()
del age_uniq["Percentage of Players"]
age_uniq.rename(columns={"Price":"Total Purchase Value","Item ID":"Purchase Count"},inplace=True)
age_uniq
age_uniq["Average Price"]= age_uniq["Total Purchase Value"]/age_uniq["Purchase Count"]
age_uniq["Normalized Avg"]= age_uniq["Total Purchase Value"]/age_uniq["Total Count"]
del age_uniq["Total Count"]
age_uniq_format= age_uniq
age_uniq_format.style.format({"Average Price": "${:.2f}", "Total Purchase Value": "${:,.2f}", "Normalized Avg":"${:.2f}"})



,Total Purchase Value,Purchase Count,Average Price,Normalized Avg
Age Ranges,,,,
10 - 14,$96.95,35,$2.77,$4.22
15 - 19,$386.42,133,$2.91,$3.86
20 - 24,$978.77,336,$2.91,$3.78
25 - 29,$370.33,125,$2.96,$4.26
30 - 34,$197.25,64,$3.08,$4.20
35 - 39,$119.40,42,$2.84,$4.42
<10,$83.46,28,$2.98,$4.39
> 40,$53.75,17,$3.16,$4.89


In [13]:
#Top Spenders
top_spend = pd.DataFrame(nerds.groupby("SN")["Item ID"].count())
top_spd_price = pd.DataFrame(nerds.groupby("SN")["Price"].sum())
spd_df=pd.merge(top_spend,top_spd_price, left_index=True, right_index=True)

spd_df.rename(columns={"SN":"Username", "Item ID":"Purchase Count","Price":"Total Purchase Value"}, inplace=True)

spd_df.sort_values("Total Purchase Value",ascending=False, inplace=True)
spd_df["Average Purchase Price"] = spd_df["Total Purchase Value"]/spd_df["Purchase Count"]
spd_df_format =spd_df
spd_df_format.style.format({"Purchase Count":"{:,0f}","Total Purchase Value": "${:,.2f}","Average Purchase Price": "${:.2f}"})
spd_df_format.head()



,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Undirrala66,5,17.06,3.412000
Saedue76,4,13.56,3.390000
Mindimnya67,4,12.74,3.185000
Haellysu29,3,12.73,4.243333
Eoda93,3,11.58,3.860000


In [14]:
#Top Items
top_item = pd.DataFrame(nerds.groupby("Item Name")["Item ID"].count())
top_item_price = pd.DataFrame(nerds.groupby("Item Name")["Price"].sum())
top_df=pd.merge(top_item,top_item_price, left_index=True, right_index=True)
top_df.sort_values("Item ID",ascending=False, inplace=True)
top_df["Average Price"]= top_df["Price"]/top_df["Item ID"]
top_df["Price"]
top_df.head()

,Item ID,Price,Average Price
Item Name,,,
Final Critic,14,38.60,2.757143
Arcane Gem,11,24.53,2.230000
"Betrayal, Whisper of Grieving Widows",11,25.85,2.350000
Stormcaller,10,34.65,3.465000
Woeful Adamantite Claymore,9,11.16,1.240000


In [15]:
#Most profitable items
top_item = pd.DataFrame(nerds.groupby("Item Name")["Item ID"].count())
top_item_price = pd.DataFrame(nerds.groupby("Item Name")["Price"].sum())
top_df=pd.merge(top_item,top_item_price, left_index=True, right_index=True)

top_df["Average Price"]= top_df["Price"]/top_df["Item ID"]
top_df["Price"]
top_df.sort_values("Average Price",ascending=False, inplace=True)
top_df.head()

,Item ID,Price,Average Price
Item Name,,,
Orenmir,6,29.70,4.95
"Winterthorn, Defender of Shifting Worlds",4,19.56,4.89
Singed Scalpel,6,29.22,4.87
Stormfury Longsword,5,24.15,4.83
The Decapitator,3,14.46,4.82
